In [15]:
import os
import json
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [16]:
# 重命名内容
def rename_columns_text(df):
    # 字段中包含'_prediction_mean','predeiction_majority',统一替换为空字符串
    df.columns = df.columns.str.replace('_prediction_mean', '')
    df.columns = df.columns.str.replace('_prediction_majority', '')
    df.columns = df.columns.str.replace('_mean', '')
    df.columns = df.columns.str.replace('_majority', '')
    # 字段中包含'irrigation','drainage',去除其他字符串只保留'irrigation','drainage'
    df.columns = df.columns.str.replace(r'.*?(irrigation|drainage).*', r'\1', regex=True)
    return df
# 计算地形部位
def calculate_slope_position(value):
    value = int(value)
    # 1 2 为山地坡下，3 4 为山地坡中，5 6 为山地坡上
    if value == 1 or value == 2:
        return '山地坡下'
    elif value == 3 or value == 4:
        return '山地坡中'
    elif value == 5 or value == 6:
        return '山地坡上'
    else:
        return None
# 计算灌溉能力
def calculate_irrigation_ability(value):
    value = int(value)
    # 1 不满足 2 基本满足 3 满足 4 充分满足
    if value == 1:
        return '不满足'
    elif value == 2:
        return '基本满足'
    elif value == 3:
        return '满足'
    elif value == 4:
        return '充分满足'
    else:
        return None
# 计算排水能力
def calculate_drainage_ability(value):
    value = int(value)
    # 1 不满足 2 基本满足 3 满足 4 充分满足
    if value == 1:
        return '不满足'
    elif value == 2:
        return '基本满足'
    elif value == 3:
        return '满足'
    elif value == 4:
        return '充分满足'
    else:
        return None
# 计算耕层质地
def calculate_soil_texture(value,search_dict):
    value = int(value)
    # 字典的值等于value的key
    for key, val in search_dict.items():
        if val == value:
            return key
    return None
# 计算质地构型
def determine_soil_texture_structure(soil_texture: str,soil_type: str, land_use: str) -> str:
    """
    根据土壤类型、土地利用方式和土壤质地判断质地构型
    
    参数:
        soil_type: 土壤类型 (黄壤、石灰土、水稻土、潮土、黄棕壤)
        land_use: 土地利用方式 (水田、旱地、水浇地)
        soil_texture: 土壤质地 (壤质黏土、黏土、黏壤土、粉(砂)质黏土、砂质黏土、
                        砂质黏壤土、重黏土、粉(砂)质黏壤土、砂质壤土、粉(砂)质壤土、壤土)
    
    返回:
        质地构型类型 (上松下紧型、海绵型、夹层型、紧实型、上紧下松型、松散型、薄层型)
    """
    # 质地分类字典
    texture_groups = {
        "黏质类": ["黏土", "重黏土", "壤质黏土"],
        "壤质类": ["黏壤土", "粉(砂)质黏壤土", "砂质黏壤土", "壤土"],
        "砂质类": ["砂质壤土", "粉(砂)质壤土"],
        "特殊类": ["粉(砂)质黏土", "砂质黏土"]
    }
    
    # 规则判断
    if land_use == "水田":
        if soil_type == "水稻土":
            return "紧实型"
        if soil_texture in texture_groups["黏质类"]:
            return "上松下紧型"
        return "海绵型"
    
    if land_use in ["旱地", "水浇地"]:
        if soil_type in ["黄壤", "黄棕壤"]:
            if soil_texture in texture_groups["砂质类"]:
                return "松散型"
            if soil_texture in texture_groups["壤质类"]:
                return "海绵型"
            return "上松下紧型"
        
        if soil_type == "石灰土":
            if soil_texture in texture_groups["黏质类"]:
                return "夹层型"
            return "薄层型"
        
        if soil_type == "潮土":
            if soil_texture in texture_groups["砂质类"] + ["壤土"]:
                return "上紧下松型"
            return "海绵型"
    
    # 默认返回（当所有条件不满足时）
    return "海绵型"  # 海绵型作为最普遍的构型


# 保存为shp
def save_shp(df,path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    # 直接创建GeoDataFrame，因为geometry已经是Shapely对象
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.to_file(path, driver='ESRI Shapefile', encoding='gbk')


In [17]:
# 数据路径
data_path = r"G:\soil_property_result\qzs\grade_evaluation\table\grade_evaluation_sample_near.csv"

In [18]:
df = pd.read_csv(data_path)
df.rename(columns={'TRZD':'邻近质地'}, inplace=True)

In [19]:
# 重命名字段
df = rename_columns_text(df)

In [20]:
# 重命名字段
map_dict = {'PH':'酸碱度','OM':'有机质','AK':'速效钾','AP':'有效磷','TRRZ':'土壤容重','GZCHD':'耕层厚度','YXTCHD':'有效土层厚度',
            'CD':'镉','HG':'汞','AS2':'砷','PB':'铅','CR':'铬','slopepostion':'坡位','dem':'海拔高度','irrigation':'灌溉','drainage':'排水','TRZD':'质地'}
df.rename(columns=map_dict, inplace=True)

In [21]:
# 计算地形部位
df['地形部位'] = df['坡位'].apply(calculate_slope_position)

In [22]:
# 计算灌溉能力
df['灌溉能力'] = df['灌溉'].apply(calculate_irrigation_ability)


In [23]:
# 计算排水能力
df['排水能力'] = df['排水'].apply(calculate_drainage_ability)

In [24]:
# 计算耕层质地（邻近点）
# dict_path = r"G:\soil_property_result\qzs\models\soil_property_class\label_mappings\TRZD_mapping.json"
# search_dict = json.load(open(dict_path, 'r', encoding='utf-8'))
# df['耕层质地'] = df['质地'].apply(calculate_soil_texture, args=(search_dict,))
# 计算耕层质地
df['耕层质地'] = df['邻近质地']

In [25]:
# 计算质地构型
df['质地构型'] = df.apply(lambda row: determine_soil_texture_structure(row['耕层质地'], row['Class1_tl'], row['DLMC']), axis=1)

In [26]:
# 整理字段
filter_df = df[['FID', 'DLMC', 'ZLDWDM', 'ZLDWMC', 'Class1', 'Class1_ts', 'Class1_yl',
       'Class1_tl', 'DLBM', 'QSDWDM', 'QSDWMC', 'DLLB', 'DLDM', 'MZMC', 'gridcode', 'Centroid_X', 'Centroid_Y',
       'project_Area','酸碱度', '有机质', '速效钾', '有效磷', '土壤容重', '耕层厚度',
       '有效土层厚度', '海拔高度', '地形部位', '灌溉能力', '排水能力', '耕层质地','质地构型']]

In [27]:
# 保证都为正值（对对应列应用ABS函数）
for col in ['酸碱度', '有机质', '速效钾', '有效磷', '土壤容重', '耕层厚度','有效土层厚度', '海拔高度']:
    filter_df.loc[:, col] = filter_df[col].apply(lambda x: abs(x))

In [28]:
# 保存为csv和excel
filter_df.to_csv(r'G:\soil_property_result\qzs\grade_evaluation\table\grade_evaluation_have_index.csv', index=False)
filter_df.to_excel(r'G:\soil_property_result\qzs\grade_evaluation\table\grade_evaluation_have_index.xlsx', index=False)